In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# Lifted sub-elliptic diffusions

In [ ]:
# SO(3)
from src.groups.SON import *
G = SON(3)
print(G)

# SO(3) acts on S^2
from src.manifolds.S2 import *
M = S2()
print(M)

from src.plotting import *

In [ ]:
from src.group import invariant_metric
invariant_metric.initialize(G)

from src.group import energy
energy.initialize(G)

from src.group.quotient import *

In [ ]:
q = np.array([1e-6,0,0])
g = G.psif(q)
v = np.array([0,1,1])
p = G.sharppsif(q,v)

In [ ]:
# sample Brownian paths on SO(3) and S^2. Note bi-invariant metric let's process descend to Brownian motion on S^2
from src.stochastics import Brownian_inv
Brownian_inv.initialize(G)

G.sigma.set_value(np.eye(G.dim.eval())) # set metric
K = 3
gss = np.zeros((K,n_steps.eval())+g.shape)
# srng.seed(422)
for i in range(K):
    (ts,gs) = G.Brownian_invf(g,dWsf(G.dim.eval()))
    gss[i] = gs

# plot samples
newfig()
for i in range(K):
    G.plotg(gss[i])
plt.show()

# on S2
x = np.array([0,0,1])
gss_M = np.zeros((K,n_steps.eval(),M.emb_dim.eval(),))
newfig()
M.plot()
for i in range(K):
    gss_M[i] = M.actsf(gss[i].transpose((1,2,0)),x).T
    M.plotx(gss_M[i])
plt.show()

In [ ]:
# truncated SO(3) Brownian motion
sde_Brownian_inv_horz = get_sde_horz(G.sde_Brownian_inv,lambda g: M.act(g,x),G,M)
Brownian_inv_horz = lambda g,dWt: integrate_sde(sde_Brownian_inv_horz,
                                                  integrator_stratonovich,
                                                  g,dWt)
g0 = G.element()
Brownian_inv_horzf = theano.function([g0,dWt], Brownian_inv_horz(g0,dWt))

# make simulations
gss = np.zeros((K,n_steps.eval())+g.shape)
vl = G.psif(v)
for l in range(K):
    (ts,gs) = Brownian_inv_horzf(vl,dWsf(G.dim.eval()))
    gss[i] = gs

# plot samples
newfig()
for i in range(K):
    G.plotg(gss[i])
plt.show()

# on S2
x = np.array([0,0,1])
gss_M = np.zeros((K,n_steps.eval(),M.emb_dim.eval(),))
newfig()
M.plot()
for i in range(K):
    gss_M[i] = M.actsf(gss[i].transpose((1,2,0)),x).T
    M.plotx(gss_M[i])
plt.show()

In [ ]:
# lift from S^2 to SO(3)

# S^2 Brownian motion
from src.Riemannian import metric
metric.initialize(M)
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

sde_Brownian_inv_lifted = get_sde_lifted(M.sde_Brownian_coords,lambda g: M.act(g,x),G,M)
Brownian_inv_lifted = lambda g,dWt: integrate_sde(sde_Brownian_inv_lifted,
                                                  integrator_stratonovich,
                                                  g,dWt)
g0 = G.element()
Brownian_inv_liftedf = theano.function([g0,dWt], Brownian_inv_lifted(g0,dWt))

# make simulations
gss = np.zeros((K,n_steps.eval())+g.shape)
vl = G.psif(v)
for l in range(K):
    (ts,gs) = Brownian_inv_horzf(vl,dWsf(G.dim.eval()))
    gss[i] = gs

# plot samples
newfig()
for i in range(K):
    G.plotg(gss[i])
plt.show()

# on S2
x = np.array([0,0,1])
gss_M = np.zeros((K,n_steps.eval(),M.emb_dim.eval(),))
newfig()
M.plot()
for i in range(K):
    gss_M[i] = M.actsf(gss[i].transpose((1,2,0)),x).T
    M.plotx(gss_M[i])
plt.show()